# Splits

## Create unpaired split
From Oulu-Casia and Casia2

In [45]:
import json
from sklearn.model_selection import train_test_split
from pathlib import Path
import random

# Specify the directories
nir_path = Path('data/CASIA_NISVIR/NIR-VIS/NIR/')
vis_path = Path('data/CASIA_NISVIR/NIR-VIS/VIS/')

# Get the contents of the directories
nir_images = list(nir_path.glob('*.bmp'))  # replace with your image extension
vis_images = list(vis_path.glob('*.jpg'))  # replace with your image extension

# Determine the size of the smaller dataset
min_size = min(len(nir_images), len(vis_images))

# Randomly select 'min_size' images from both NIR and VIS datasets
nir_images_selected = random.sample(nir_images, min_size)
vis_images_selected = random.sample(vis_images, min_size)

# Get the images that are not selected
nir_images_unselected = list(set(nir_images) - set(nir_images_selected))
vis_images_unselected = list(set(vis_images) - set(vis_images_selected))

# Split the NIR images into train, val, and test sets
nir_train_val, nir_test = train_test_split(nir_images_selected, test_size=0.2, random_state=42)
nir_train, nir_val = train_test_split(nir_train_val, test_size=0.25, random_state=42)

# Split the VIS images into train, val, and test sets
vis_train_val, vis_test = train_test_split(vis_images_selected, test_size=0.2, random_state=42)
vis_train, vis_val = train_test_split(vis_train_val, test_size=0.25, random_state=42)

# Convert the Path objects to strings
nir_train = [str(path) for path in nir_train]
nir_val = [str(path) for path in nir_val]
nir_test = [str(path) for path in nir_test]
vis_train = [str(path) for path in vis_train]
vis_val = [str(path) for path in vis_val]
vis_test = [str(path) for path in vis_test]
nir_images_unselected = [str(path) for path in nir_images_unselected]
vis_images_unselected = [str(path) for path in vis_images_unselected]

# Prepare the metadata
metadata = {
    'nir_images_selected': len(nir_images_selected),
    'vis_images_selected': len(vis_images_selected),
    'nir_images_unselected': len(nir_images_unselected),
    'vis_images_unselected': len(vis_images_unselected),
    'nir_train': len(nir_train),
    'nir_val': len(nir_val),
    'nir_test': len(nir_test),
    'vis_train': len(vis_train),
    'vis_val': len(vis_val),
    'vis_test': len(vis_test),
}

# Prepare the data to be stored in the JSON file
data = {
    'nir_train': nir_train,
    'nir_val': nir_val,
    'nir_test': nir_test,
    'nir_rest': nir_images_unselected,
    'vis_train': vis_train,
    'vis_val': vis_val,
    'vis_test': vis_test,
    'vis_rest': vis_images_unselected,
    'metadata': metadata,
}

# Write the data to the JSON file
with open('splits/new/casia2_data_splits.json', 'w') as f:
    json.dump(data, f)


In [48]:
from skeleton.data.splitter import DatasetSplitter
import json
import os

# define splitter
splitter = DatasetSplitter(
    vl_data_path=Path('data/B_OriginalImg/VL/Strong/'),
    ni_data_path=Path('data/B_OriginalImg/NI/Strong/'),
    train_n_img_picked=5,
    test_n_img_picked=2,
    val_n_img_picked=2,
    json_train_split_pth=Path('_train_tmp.json'),
    json_test_split_pth=Path('_test_tmp.json'),
    json_val_split_pth=Path('_val_tmp.json'),
)

# split the files
splitter()


# Load the temporary JSON files
with open('_train_tmp.json', 'r') as f:
    train_data = json.load(f)
with open('_test_tmp.json', 'r') as f:
    test_data = json.load(f)
with open('_val_tmp.json', 'r') as f:
    val_data = json.load(f)

# Prepare the metadata
metadata = {
    'nir_images': len(train_data['ni']) + len(test_data['ni']) + len(val_data['ni']),
    'vis_images': len(train_data['vl']) + len(test_data['vl']) + len(val_data['vl']),
    'nir_train': len(train_data['ni']),
    'nir_val': len(val_data['ni']),
    'nir_test': len(test_data['ni']),
    'vis_train': len(train_data['vl']),
    'vis_val': len(val_data['vl']),
    'vis_test': len(test_data['vl']),
}

# Prepare the data to be stored in the JSON file
data = {
    'nir_train': train_data['ni'],
    'nir_val': val_data['ni'],
    'nir_test': test_data['ni'],
    'vis_train': train_data['vl'],
    'vis_val': val_data['vl'],
    'vis_test': test_data['vl'],
    'metadata': metadata,
}

# Write the data to the JSON file
with open('splits/new/oulucasia_data_splits.json', 'w') as f:
    json.dump(data, f)

# Remove the temporary files
os.remove('_train_tmp.json')
os.remove('_test_tmp.json')
os.remove('_val_tmp.json')


In [49]:
# Load the existing data from the two files
with open('splits/new/casia2_data_splits.json', 'r') as f:
    casia2_data = json.load(f)
with open('splits/new/oulucasia_data_splits.json', 'r') as f:
    oulucasia_data = json.load(f)

# Merge the data
merged_data = {}
for key in set(casia2_data.keys()).union(oulucasia_data.keys()):
    if key != 'metadata':
        merged_data[key] = casia2_data.get(key, []) + oulucasia_data.get(key, [])

# Merge the metadata
merged_metadata = {}
for key in set(casia2_data['metadata'].keys()).union(oulucasia_data['metadata'].keys()):
    merged_metadata[key] = casia2_data['metadata'].get(key, 0) + oulucasia_data['metadata'].get(key, 0)

# Add the merged metadata to the merged data
merged_data['metadata'] = merged_metadata

# Save the merged data back to the file
with open('splits/new/merged_data_splits.json', 'w') as f:
    json.dump(merged_data, f)

## Preprocess

In [1]:
import json
import pathlib
import os
import click
import cv2

from deepface import DeepFace


class FacePreprocessor:
    def __init__(
        self,
        train_split_pth,
        test_split_pth,
        val_split_pth,
        new_train_vl_pth,
        new_train_ni_pth,
        new_test_vl_pth,
        new_test_ni_pth,
        new_val_ni_pth,
        new_val_vl_pth,
        detector_backend,
        target_size,
        new_train_split_pth=None,
        new_test_split_pth=None,
        new_val_split_pth=None,
    ):
        self.train_split_pth = train_split_pth
        self.test_split_pth = test_split_pth
        self.val_split_pth = val_split_pth
        self.new_train_vl_pth = new_train_vl_pth
        self.new_train_ni_pth = new_train_ni_pth
        self.new_test_vl_pth = new_test_vl_pth
        self.new_test_ni_pth = new_test_ni_pth
        self.new_val_ni_pth = new_val_ni_pth
        self.new_val_vl_pth = new_val_vl_pth
        self.detector_backend = detector_backend
        self.target_size = target_size
        self.new_train_split = new_train_split_pth
        self.new_test_split = new_test_split_pth
        self.new_val_split = new_val_split_pth

    def detect_and_align_face(self, image_fp):
        try:
            face_objs = DeepFace.extract_faces(
                img_path=image_fp,
                target_size=self.target_size,
                detector_backend=self.detector_backend,
                enforce_detection=False,
            )
        except Exception as e:
            print(f"ERROR at {image_fp}", e)
            return None

        if len(face_objs) != 1:
            print("NOT FOUND OR MULTIPLE FACES!")
            return None

        face = face_objs[0]["face"]

        return face

    def preprocess_part(self, fps, target_fp, spectra):
        # prepare filepath
        os.makedirs(target_fp, exist_ok=True)

        # align faces for all images
        i = 0
        preprocessed_fps = []
        for fp in fps:
            new_filename = "-".join(pathlib.PurePath(fp).parts[-3:])
            target_path = os.path.join(target_fp, new_filename)

            aligned_face = self.detect_and_align_face(fp)

            if aligned_face is None:
                continue

            aligned_face = 255 * aligned_face[:, :, ::-1]

            cv2.imwrite(target_path, aligned_face)

            print(f"#{i} {spectra} Stored: {new_filename}")
            i += 1
            preprocessed_fps.append(target_path)

        return preprocessed_fps

    def preprocess_split(self, split_pth, new_vl_path, new_ni_pth):
        with open(split_pth, "r") as f:
            paths = json.load(f)

        vl_preproc_fps = self.preprocess_part(paths["vl"], new_vl_path, "vl")
        ni_preproc_fps = self.preprocess_part(paths["ni"], new_ni_pth, "ni")

        return {"vl": vl_preproc_fps, "ni": ni_preproc_fps}

    def preprocess(self):
#         preprocess train split
        train_fps = self.preprocess_split(
            self.train_split_pth, self.new_train_vl_pth, self.new_train_ni_pth
        )

        if self.new_train_split:
            with open(self.new_train_split, "w") as f:
                json.dump(train_fps, f)

        # preprocess test split
        test_fps = self.preprocess_split(
            self.test_split_pth, self.new_test_vl_pth, self.new_test_ni_pth
        )

        if self.new_test_split:
            with open(self.new_test_split, "w") as f:
                json.dump(test_fps, f)

        # preprocess val split
        val_fps = self.preprocess_split(
            self.val_split_pth, self.new_val_vl_pth, self.new_val_ni_pth
        )

        if self.new_val_split:
            with open(self.new_val_split, "w") as f:
                json.dump(val_fps, f)

2023-12-07 00:16:14.276607: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 00:16:17.042673: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 00:16:17.059026: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 00:16:25.233904: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
with open('splits/new/merged_data_splits.json') as f:
    merged_data = json.load(f)

In [3]:
with open('_train_split_tmp.json', 'w') as f:
    json.dump({'ni': merged_data['nir_train'], 'vl': merged_data['vis_train']}, f)
with open('_test_split_tmp.json', 'w') as f:
    json.dump({'ni': merged_data['nir_test'], 'vl': merged_data['vis_test']}, f)
with open('_val_split_tmp.json', 'w') as f:
    json.dump({'ni': merged_data['nir_val'], 'vl': merged_data['vis_val']}, f)

preprocessor = FacePreprocessor(
    train_split_pth='_train_split_tmp.json',
    test_split_pth='_test_split_tmp.json',
    val_split_pth='_val_split_tmp.json',
    new_train_vl_pth='data/casia-oulucasia-unpaired/A/train',
    new_train_ni_pth='data/casia-oulucasia-unpaired/B/train',
    new_test_vl_pth='data/casia-oulucasia-unpaired/A/test',
    new_test_ni_pth='data/casia-oulucasia-unpaired/B/test',
    new_val_vl_pth='data/casia-oulucasia-unpaired/A/val',
    new_val_ni_pth='data/casia-oulucasia-unpaired/B/val',
    detector_backend='retinaface',
    target_size=(224, 224),
    new_train_split_pth='splits/new/preproc-merged-train',
    new_test_split_pth='splits/new/preproc-merged-test',
    new_val_split_pth='splits/new/preproc-merged-val',
)

preprocessor.preprocess()
               
os.remove('_train_split_tmp.json')
os.remove('_test_split_tmp.json')
os.remove('_val_split_tmp.json')

#0 vl Stored: NIR-VIS-VIS-s1_VIS_00164_005.jpg
#1 vl Stored: NIR-VIS-VIS-s1_VIS_00027_003.jpg
#2 vl Stored: NIR-VIS-VIS-s1_VIS_00023_002.jpg
#3 vl Stored: NIR-VIS-VIS-s1_VIS_00107_006.jpg
#4 vl Stored: NIR-VIS-VIS-s3_VIS_20407_003.jpg
#5 vl Stored: NIR-VIS-VIS-s3_VIS_20486_005.jpg
#6 vl Stored: NIR-VIS-VIS-s1_VIS_00032_002.jpg
#7 vl Stored: NIR-VIS-VIS-s2_VIS_10186_005.jpg
#8 vl Stored: NIR-VIS-VIS-s1_VIS_00188_008.jpg
#9 vl Stored: NIR-VIS-VIS-s3_VIS_20497_005.jpg
#10 vl Stored: NIR-VIS-VIS-s3_VIS_20445_006.jpg
#11 vl Stored: NIR-VIS-VIS-s1_VIS_00041_002.jpg
#12 vl Stored: NIR-VIS-VIS-s1_VIS_00031_010.jpg
#13 vl Stored: NIR-VIS-VIS-s1_VIS_00085_010.jpg
#14 vl Stored: NIR-VIS-VIS-s3_VIS_20406_006.jpg
#15 vl Stored: NIR-VIS-VIS-s1_VIS_00011_009.jpg
#16 vl Stored: NIR-VIS-VIS-s2_VIS_10131_002.jpg
#17 vl Stored: NIR-VIS-VIS-s1_VIS_00163_010.jpg
#18 vl Stored: NIR-VIS-VIS-s1_VIS_00167_009.jpg
#19 vl Stored: NIR-VIS-VIS-s2_VIS_10293_004.jpg
#20 vl Stored: NIR-VIS-VIS-s1_VIS_00039_003.jpg
#2

#170 vl Stored: NIR-VIS-VIS-s1_VIS_00009_003.jpg
#171 vl Stored: NIR-VIS-VIS-s1_VIS_00005_001.jpg
#172 vl Stored: NIR-VIS-VIS-s2_VIS_10272_003.jpg
#173 vl Stored: NIR-VIS-VIS-s1_VIS_00043_010.jpg
#174 vl Stored: NIR-VIS-VIS-s4_VIS_30778_002.jpg
#175 vl Stored: NIR-VIS-VIS-s3_VIS_20409_001.jpg
#176 vl Stored: NIR-VIS-VIS-s2_VIS_10251_004.jpg
#177 vl Stored: NIR-VIS-VIS-s4_VIS_30773_004.jpg
#178 vl Stored: NIR-VIS-VIS-s2_VIS_10160_006.jpg
#179 vl Stored: NIR-VIS-VIS-s2_VIS_10284_003.jpg
#180 vl Stored: NIR-VIS-VIS-s2_VIS_00164_004.jpg
#181 vl Stored: NIR-VIS-VIS-s2_VIS_10284_004.jpg
#182 vl Stored: NIR-VIS-VIS-s2_VIS_10140_005.jpg
#183 vl Stored: NIR-VIS-VIS-s1_VIS_00185_009.jpg
#184 vl Stored: NIR-VIS-VIS-s3_VIS_20430_005.jpg
#185 vl Stored: NIR-VIS-VIS-s1_VIS_00106_010.jpg
#186 vl Stored: NIR-VIS-VIS-s2_VIS_10180_001.jpg
#187 vl Stored: NIR-VIS-VIS-s2_VIS_10203_005.jpg
#188 vl Stored: NIR-VIS-VIS-s2_VIS_10039_006.jpg
#189 vl Stored: NIR-VIS-VIS-s2_VIS_10113_002.jpg
#190 vl Stored: NIR-

#337 vl Stored: NIR-VIS-VIS-s1_VIS_00182_010.jpg
#338 vl Stored: NIR-VIS-VIS-s1_VIS_00106_004.jpg
#339 vl Stored: NIR-VIS-VIS-s2_VIS_10267_006.jpg
#340 vl Stored: NIR-VIS-VIS-s1_VIS_00050_004.jpg
#341 vl Stored: NIR-VIS-VIS-s2_VIS_10266_001.jpg
#342 vl Stored: NIR-VIS-VIS-s2_VIS_10095_006.jpg
#343 vl Stored: NIR-VIS-VIS-s1_VIS_00001_012.jpg
#344 vl Stored: NIR-VIS-VIS-s3_VIS_20333_001.jpg
#345 vl Stored: NIR-VIS-VIS-s2_VIS_10233_004.jpg
#346 vl Stored: NIR-VIS-VIS-s2_VIS_10232_005.jpg
#347 vl Stored: NIR-VIS-VIS-s1_VIS_00202_010.jpg
#348 vl Stored: NIR-VIS-VIS-s2_VIS_10224_002.jpg
#349 vl Stored: NIR-VIS-VIS-s1_VIS_00149_008.jpg
#350 vl Stored: NIR-VIS-VIS-s1_VIS_00051_003.jpg
#351 vl Stored: NIR-VIS-VIS-s2_VIS_10131_003.jpg
#352 vl Stored: NIR-VIS-VIS-s1_VIS_00072_007.jpg
#353 vl Stored: NIR-VIS-VIS-s2_VIS_00071_003.jpg
#354 vl Stored: NIR-VIS-VIS-s3_VIS_20389_006.jpg
#355 vl Stored: NIR-VIS-VIS-s3_VIS_20463_005.jpg
#356 vl Stored: NIR-VIS-VIS-s2_VIS_10220_006.jpg
#357 vl Stored: NIR-

#504 vl Stored: NIR-VIS-VIS-s3_VIS_20429_006.jpg
#505 vl Stored: NIR-VIS-VIS-s3_VIS_20392_005.jpg
#506 vl Stored: NIR-VIS-VIS-s2_VIS_10188_006.jpg
#507 vl Stored: NIR-VIS-VIS-s1_VIS_00086_005.jpg
#508 vl Stored: NIR-VIS-VIS-s1_VIS_00108_002.jpg
#509 vl Stored: NIR-VIS-VIS-s1_VIS_00054_006.jpg
#510 vl Stored: NIR-VIS-VIS-s2_VIS_10133_004.jpg
#511 vl Stored: NIR-VIS-VIS-s2_VIS_10253_006.jpg
#512 vl Stored: NIR-VIS-VIS-s1_VIS_00166_010.jpg
#513 vl Stored: NIR-VIS-VIS-s2_VIS_10187_002.jpg
#514 vl Stored: NIR-VIS-VIS-s1_VIS_00121_008.jpg
#515 vl Stored: NIR-VIS-VIS-s1_VIS_00151_013.jpg
#516 vl Stored: NIR-VIS-VIS-s1_VIS_00079_006.jpg
#517 vl Stored: NIR-VIS-VIS-s3_VIS_20401_002.jpg
#518 vl Stored: NIR-VIS-VIS-s3_VIS_20463_003.jpg
#519 vl Stored: NIR-VIS-VIS-s2_VIS_10087_002.jpg
#520 vl Stored: NIR-VIS-VIS-s1_VIS_00019_004.jpg
#521 vl Stored: NIR-VIS-VIS-s2_VIS_10184_004.jpg
#522 vl Stored: NIR-VIS-VIS-s2_VIS_10036_002.jpg
#523 vl Stored: NIR-VIS-VIS-s2_VIS_10077_005.jpg
#524 vl Stored: NIR-

#671 vl Stored: NIR-VIS-VIS-s3_VIS_20430_002.jpg
#672 vl Stored: NIR-VIS-VIS-s3_VIS_20365_004.jpg
#673 vl Stored: NIR-VIS-VIS-s2_VIS_10089_005.jpg
#674 vl Stored: NIR-VIS-VIS-s3_VIS_20489_004.jpg
#675 vl Stored: NIR-VIS-VIS-s1_VIS_00014_002.jpg
#676 vl Stored: NIR-VIS-VIS-s1_VIS_00115_004.jpg
#677 vl Stored: NIR-VIS-VIS-s1_VIS_00184_007.jpg
#678 vl Stored: NIR-VIS-VIS-s2_VIS_10124_002.jpg
#679 vl Stored: NIR-VIS-VIS-s1_VIS_00035_008.jpg
#680 vl Stored: NIR-VIS-VIS-s2_VIS_10226_004.jpg
#681 vl Stored: NIR-VIS-VIS-s3_VIS_20390_005.jpg
#682 vl Stored: NIR-VIS-VIS-s1_VIS_00188_006.jpg
#683 vl Stored: NIR-VIS-VIS-s2_VIS_10121_007.jpg
#684 vl Stored: NIR-VIS-VIS-s3_VIS_20454_004.jpg
#685 vl Stored: NIR-VIS-VIS-s1_VIS_00065_003.jpg
#686 vl Stored: NIR-VIS-VIS-s2_VIS_10054_003.jpg
#687 vl Stored: NIR-VIS-VIS-s2_VIS_10142_006.jpg
#688 vl Stored: NIR-VIS-VIS-s1_VIS_00201_006.jpg
#689 vl Stored: NIR-VIS-VIS-s1_VIS_00070_007.jpg
#690 vl Stored: NIR-VIS-VIS-s1_VIS_00097_006.jpg
#691 vl Stored: NIR-

...

#972 ni Stored: NIR-VIS-NIR-s1_NIR_00195_013.bmp
#973 ni Stored: NIR-VIS-NIR-s2_NIR_10048_016.bmp
#974 ni Stored: NIR-VIS-NIR-s2_NIR_10230_014.bmp
#975 ni Stored: NIR-VIS-NIR-s2_NIR_10023_015.bmp
#976 ni Stored: NIR-VIS-NIR-s2_NIR_10060_012.bmp
#977 ni Stored: NIR-VIS-NIR-s3_NIR_20460_013.bmp
#978 ni Stored: NIR-VIS-NIR-s1_NIR_00089_011.bmp
#979 ni Stored: NIR-VIS-NIR-s1_NIR_00081_007.bmp
#980 ni Stored: NIR-VIS-NIR-s2_NIR_10022_003.bmp
#981 ni Stored: NIR-VIS-NIR-s2_NIR_10261_018.bmp
#982 ni Stored: NIR-VIS-NIR-s3_NIR_20338_003.bmp
#983 ni Stored: NIR-VIS-NIR-s1_NIR_00149_007.bmp
#984 ni Stored: NIR-VIS-NIR-s1_NIR_00067_003.bmp
#985 ni Stored: NIR-VIS-NIR-s4_NIR_30756_016.bmp
#986 ni Stored: NIR-VIS-NIR-s2_NIR_10099_009.bmp
#987 ni Stored: NIR-VIS-NIR-s1_NIR_00190_025.bmp
#988 ni Stored: NIR-VIS-NIR-s1_NIR_00075_001.bmp
#989 ni Stored: NIR-VIS-NIR-s3_NIR_20338_014.bmp
#990 ni Stored: NIR-VIS-NIR-s2_NIR_10135_005.bmp
#991 ni Stored: NIR-VIS-NIR-s1_NIR_00200_016.bmp
#992 ni Stored: NIR-

#1174 ni Stored: P072-Anger-019.jpeg
#1175 ni Stored: P072-Surprise-011.jpeg
#1176 ni Stored: P072-Surprise-002.jpeg
#1177 ni Stored: P072-Disgust-011.jpeg
#1178 ni Stored: P072-Disgust-010.jpeg
#1179 ni Stored: P041-Happiness-014.jpeg
#1180 ni Stored: P041-Happiness-021.jpeg
#1181 ni Stored: P041-Sadness-004.jpeg
#1182 ni Stored: P041-Sadness-007.jpeg
#1183 ni Stored: P041-Fear-004.jpeg
#1184 ni Stored: P041-Fear-003.jpeg
#1185 ni Stored: P041-Anger-005.jpeg
#1186 ni Stored: P041-Anger-003.jpeg
#1187 ni Stored: P041-Surprise-014.jpeg
#1188 ni Stored: P041-Surprise-010.jpeg
#1189 ni Stored: P041-Disgust-012.jpeg
#1190 ni Stored: P041-Disgust-005.jpeg
#1191 ni Stored: P055-Happiness-009.jpeg
#1192 ni Stored: P055-Happiness-007.jpeg
#1193 ni Stored: P055-Sadness-000.jpeg
#1194 ni Stored: P055-Sadness-010.jpeg
#1195 ni Stored: P055-Fear-017.jpeg
#1196 ni Stored: P055-Fear-011.jpeg
#1197 ni Stored: P055-Anger-014.jpeg
#1198 ni Stored: P055-Anger-006.jpeg
#1199 ni Stored: P055-Surprise-009.

#1386 ni Stored: P003-Sadness-005.jpeg
#1387 ni Stored: P003-Fear-001.jpeg
#1388 ni Stored: P003-Fear-002.jpeg
#1389 ni Stored: P003-Anger-009.jpeg
#1390 ni Stored: P003-Anger-002.jpeg
#1391 ni Stored: P003-Surprise-001.jpeg
#1392 ni Stored: P003-Surprise-011.jpeg
#1393 ni Stored: P003-Disgust-005.jpeg
#1394 ni Stored: P003-Disgust-003.jpeg
#1395 ni Stored: P042-Happiness-005.jpeg
#1396 ni Stored: P042-Happiness-016.jpeg
#1397 ni Stored: P042-Sadness-015.jpeg
#1398 ni Stored: P042-Sadness-019.jpeg
#1399 ni Stored: P042-Fear-013.jpeg
#1400 ni Stored: P042-Fear-003.jpeg
#1401 ni Stored: P042-Anger-000.jpeg
#1402 ni Stored: P042-Anger-005.jpeg
#1403 ni Stored: P042-Surprise-012.jpeg
#1404 ni Stored: P042-Surprise-008.jpeg
#1405 ni Stored: P042-Disgust-000.jpeg
#1406 ni Stored: P042-Disgust-006.jpeg
#1407 ni Stored: P012-Happiness-012.jpeg
#1408 ni Stored: P012-Happiness-003.jpeg
#1409 ni Stored: P012-Sadness-013.jpeg
#1410 ni Stored: P012-Sadness-006.jpeg
#1411 ni Stored: P012-Fear-019.jp

#1599 ni Stored: P005-Happiness-001.jpeg
#1600 ni Stored: P005-Happiness-007.jpeg
#1601 ni Stored: P005-Sadness-001.jpeg
#1602 ni Stored: P005-Sadness-017.jpeg
#1603 ni Stored: P005-Fear-004.jpeg
#1604 ni Stored: P005-Fear-015.jpeg
#1605 ni Stored: P005-Anger-012.jpeg
#1606 ni Stored: P005-Anger-002.jpeg
#1607 ni Stored: P005-Surprise-014.jpeg
#1608 ni Stored: P005-Surprise-006.jpeg
#1609 ni Stored: P005-Disgust-013.jpeg
#1610 ni Stored: P005-Disgust-010.jpeg
#1611 ni Stored: P023-Happiness-001.jpeg
#1612 ni Stored: P023-Happiness-016.jpeg
#1613 ni Stored: P023-Sadness-001.jpeg
#1614 ni Stored: P023-Sadness-006.jpeg
#1615 ni Stored: P023-Fear-001.jpeg
#1616 ni Stored: P023-Fear-002.jpeg
#1617 ni Stored: P023-Anger-012.jpeg
#1618 ni Stored: P023-Anger-002.jpeg
#1619 ni Stored: P023-Surprise-000.jpeg
#1620 ni Stored: P023-Surprise-011.jpeg
#1621 ni Stored: P023-Disgust-028.jpeg
#1622 ni Stored: P023-Disgust-020.jpeg
#1623 ni Stored: P019-Happiness-001.jpeg
#1624 ni Stored: P019-Happiness

#1811 ni Stored: P034-Surprise-014.jpeg
#1812 ni Stored: P034-Surprise-013.jpeg
#1813 ni Stored: P034-Disgust-016.jpeg
#1814 ni Stored: P034-Disgust-021.jpeg
#1815 ni Stored: P066-Happiness-011.jpeg
#1816 ni Stored: P066-Happiness-003.jpeg
#1817 ni Stored: P066-Sadness-014.jpeg
#1818 ni Stored: P066-Sadness-015.jpeg
#1819 ni Stored: P066-Fear-000.jpeg
#1820 ni Stored: P066-Fear-007.jpeg
#1821 ni Stored: P066-Anger-009.jpeg
#1822 ni Stored: P066-Anger-016.jpeg
#1823 ni Stored: P066-Surprise-014.jpeg
#1824 ni Stored: P066-Surprise-010.jpeg
#1825 ni Stored: P066-Disgust-011.jpeg
#1826 ni Stored: P066-Disgust-015.jpeg
#1827 ni Stored: P050-Happiness-009.jpeg
#1828 ni Stored: P050-Happiness-027.jpeg
#1829 ni Stored: P050-Sadness-000.jpeg
#1830 ni Stored: P050-Sadness-002.jpeg
#1831 ni Stored: P050-Fear-001.jpeg
#1832 ni Stored: P050-Fear-010.jpeg
#1833 ni Stored: P050-Anger-014.jpeg
#1834 ni Stored: P050-Anger-006.jpeg
#1835 ni Stored: P050-Surprise-019.jpeg
#1836 ni Stored: P050-Surprise-0

In [55]:
merged_data.keys()

dict_keys(['nir_train', 'vis_test', 'nir_test', 'vis_train', 'nir_rest', 'nir_val', 'vis_val', 'vis_rest', 'metadata'])